In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pycocotools

In [ ]:
!pip install opencv-python

In [ ]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import os
import cv2
from tqdm import tqdm
import random


#initial setup


In [ ]:
#After downloading the zip files of the ikages from the dataset, this code just helps is unpavking all of them and the json file ofr annoatations.

In [ ]:
%cd /content/drive/MyDrive/TBBR

get annotations

In [ ]:
#!wget -r https://zenodo.org/records/7022736/files/Flug1_100-104Media_coco.json?download=1 -P /content/drive/MyDrive/TBBR

In [ ]:
#!unzip /content/drive/MyDrive/TBBR/zenodo.org/records/7360996/files/Flug_100.zip?download=1 -d /content/drive/MyDrive/TBBR

In [ ]:
#df = pd.read_json
with open('/content/drive/MyDrive/TBBR/train/Flug1_100-104Media_coco.json', 'r') as f:
  data = json.load(f)



extract the images

In [ ]:
!apt-get install -y zstd

In [ ]:
!zstd -d /content/drive/MyDrive/TBBR/train/Flug1_100.tar.zst --stdout | tar -xvf - -C /content/drive/MyDrive/TBBR/train

separate annotation of firts 100

In [ ]:
#dont run
import json

# Load the JSON file
input_file_path = '/content/drive/MyDrive/TBBR/train/Flug1_100-104Media_coco.json'  # Replace with the actual path to your JSON file
output_file_path = '/content/drive/MyDrive/TBBR/train/Flug1_100Media_coco.json'  # Output file name

# Read the data from the original JSON file
with open(input_file_path, 'r') as file:
    data = json.load(file)

# Step 1: Filter images to include only those that belong to Flug1_100
flug100_images = [img for img in data.get("images", []) if 'Flug1_100/' in img.get("file_name", "")]

# Step 2: Get the image IDs for Flug1_100 images
flug100_image_ids = {img["id"] for img in flug100_images}

# Step 3: Filter annotations that correspond to Flug1_100 images
flug100_annotations = [ann for ann in data.get("annotations", []) if ann["image_id"] in flug100_image_ids]

# Step 4: Prepare the new JSON data structure with filtered images, annotations, and original categories
filtered_data = {
    "images": flug100_images,
    "annotations": flug100_annotations,
    "categories": data.get("categories", [])  # Categories can be kept as they are
}

# Step 5: Save the filtered data to a new JSON file
with open(output_file_path, 'w') as output_file:
    json.dump(filtered_data, output_file)

print(f"Filtered JSON file saved to {output_file_path}")

Filtered JSON file saved to /content/drive/MyDrive/TBBR/train/Flug1_100Media_coco.json


#visualization

In [ ]:
with open('/content/drive/MyDrive/TBBR/train/Flug1_100Media_coco.json', 'r') as f:
  data1 = json.load(f)


In [ ]:
print(data1)

In [ ]:
image_path = "/content/drive/MyDrive/TBBR/train/images/Flug1_100/DJI_0055_R.npy"  # Replace with your file path
image = np.load(image_path)  # Shape: (2680, 3370, 5)
print(f"Number of channels: {image.shape[2]}")

# Extract RGB (BGR) channels and convert to RGB format
b_channel = image[:, :, 0]
g_channel = image[:, :, 1]
r_channel = image[:, :, 2]

# Stack to form an RGB image (H, W, 3)
rgb_image = np.stack([r_channel, g_channel, b_channel], axis=-1)

# Normalize RGB image (if pixel values > 1)
rgb_image = rgb_image / np.max(rgb_image)

# Extract the Thermal channel
thermal_image = image[:, :, 3]  # Thermal is the 4th channel

# Normalize the thermal image for better visualization
thermal_image_normalized = (thermal_image - np.min(thermal_image)) / (np.max(thermal_image) - np.min(thermal_image))


#depth channel
depth_image = image[:, :, 4]
depth_image_normalized = (depth_image - np.min(depth_image)) / (np.max(depth_image) - np.min(depth_image))

# Plot the RGB and Thermal images
plt.figure(figsize=(15, 10))

# Plot RGB image
plt.subplot(1, 3, 1)
plt.imshow(rgb_image)
plt.title('RGB Image')
plt.axis('off')

# Plot Thermal image
plt.subplot(1, 3, 2)
plt.imshow(thermal_image_normalized, cmap='hot')  # Use 'hot' for thermal visualization
plt.title('Thermal Image')
plt.axis('off')

#depth
# Plot Depth image
plt.subplot(1, 3, 3)
plt.imshow(depth_image_normalized, cmap='viridis')  # 'viridis' is good for depth visualization
plt.title('Depth Image')
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Load the COCO JSON file
annotation_file = "/content/drive/MyDrive/TBBR/train/Flug1_100Media_coco.json"
coco = COCO(annotation_file)



# Select an image by its ID (pick the first one as an example)
image_id = list(coco.imgs.keys())[4]  # Replace with desired image ID
image_info = coco.loadImgs(image_id)[0]



# Load the corresponding .npy image
image_path = f"/content/drive/MyDrive/TBBR/train/images/{image_info['file_name']}"  # Adjust path as needed
image = np.load(image_path)  # Shape: (2680, 3370, 5)

# Extract RGB channels
b_channel, g_channel, r_channel = image[:, :, 0], image[:, :, 1], image[:, :, 2]
rgb_image = np.stack([r_channel, g_channel, b_channel], axis=-1) / np.max(image[:, :, :3])

thermal_image = image[:, :, 3]

# Normalize the Thermal channel for visualization
thermal_image_normalized = (thermal_image - np.min(thermal_image)) / (np.max(thermal_image) - np.min(thermal_image))

# Get annotations for the image
ann_ids = coco.getAnnIds(imgIds=image_id)
annotations = coco.loadAnns(ann_ids)

# Plot the image with annotations
plt.figure(figsize=(10, 10))
plt.imshow(thermal_image_normalized)
plt.axis('off')

# Overlay annotations
for ann in annotations:
    # Draw bounding box (if present)
    if 'bbox' in ann:
        x, y, w, h = ann['bbox']
        plt.gca().add_patch(plt.Rectangle((x, y), w, h, edgecolor='red', facecolor='none', lw=2))

    # Draw segmentation mask (if present)
    if 'segmentation' in ann:
        for seg in ann['segmentation']:
            seg_np = np.array(seg).reshape(-1, 2)
            plt.gca().add_patch(plt.Polygon(seg_np, color='blue', alpha=0.3))

plt.title(f"Image ID: {image_id} with Annotations")
plt.show()

#preprocessing

In [ ]:
#dont run
# Load annotations
annotation_path = "/content/drive/MyDrive/TBBR/train/Flug1_100Media_coco.json"
with open(annotation_path, 'r') as f:
    coco_data = json.load(f)

# Augmentation Functions
def random_flip(image, annotations, flip_prob=0.5):
    """Randomly flips an image horizontally and adjusts annotations."""
    if random.random() < flip_prob:
        # Flip image horizontally
        image = np.flip(image, axis=1).copy()

        # Adjust bounding boxes
        for ann in annotations:
            x, y, w, h = ann['bbox']
            ann['bbox'][0] = image.shape[1] - x - w

        # Adjust segmentation masks
        for ann in annotations:
            if 'segmentation' in ann:
                for seg in ann['segmentation']:
                    for i in range(0, len(seg), 2):
                        seg[i] = image.shape[1] - seg[i]

    return image, annotations


# Process each image
for img_info in tqdm(coco_data['images']):
    image_path = f"/content/drive/MyDrive/TBBR/train/images/{img_info['file_name']}"
    image = np.load(image_path)

    #denoising
    image[:, :, 3] = cv2.GaussianBlur(image[:, :, 3], (5, 5), 0)


    # Normalize
    for i in range(5):  # Normalize all channels
        image[:, :, i] = (image[:, :, i] - image[:, :, i].min()) / (image[:, :, i].max() - image[:, :, i].min())

    # Get annotations for this image
    annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] == img_info['id']]

    # Apply augmentations
    image, annotations = random_flip(image, annotations)


    # Resize (optional)
    scale_x = 0.5
    scale_y = 0.5
    resized_image = cv2.resize(image, (int(image.shape[1] * scale_x), int(image.shape[0] * scale_y)))

    # Update annotations
    for ann in coco_data['annotations']:
        if ann['image_id'] == img_info['id']:
            x, y, w, h = ann['bbox']
            ann['bbox'] = [x * scale_x, y * scale_y, w * scale_x, h * scale_y]

            if 'segmentation' in ann:
                ann['segmentation'] = [[coord * scale_x if i % 2 == 0 else coord * scale_y for i, coord in enumerate(seg)] for seg in ann['segmentation']]

    # Save augmented and preprocessed image
    np.save(f"/content/drive/MyDrive/TBBR/train/images/{img_info['file_name']}", image)



# Save updated annotations
with open("/content/drive/MyDrive/TBBR/train/Flug1_100Media_coco.json", 'w') as f:
    json.dump(coco_data, f)
